<a href="https://colab.research.google.com/github/sharjeeljafri/options-greek-calculator/blob/main/Option_Greeks_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
from scipy.stats import norm

def validate_inputs(S, K, T, r, sigma, option_type):
    """Validate user inputs for Black-Scholes calculation."""
    if not all(isinstance(x, (int, float)) for x in [S, K, T, r, sigma]):
        raise ValueError("All numeric inputs must be numbers.")
    if S <= 0 or K <= 0 or T <= 0 or sigma <= 0:
        raise ValueError("Prices, time, and volatility must be positive.")
    if option_type.lower() not in ["call", "put"]:
        raise ValueError("Option type must be 'call' or 'put'.")
    return True

def black_scholes_greeks(S, K, T, r, sigma, option_type="call"):
    """Calculate Black-Scholes option price and Greeks."""
    validate_inputs(S, K, T, r, sigma, option_type)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type.lower() == "call":
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        delta = norm.cdf(d1)
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)) / 365
        rho = K * T * np.exp(-r * T) * norm.cdf(d2) / 100
    else:  # put
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        delta = norm.cdf(d1) - 1
        theta = (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)) / 365
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100

    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100

    return {"Price": price, "Delta": delta, "Gamma": gamma, "Theta": theta, "Vega": vega, "Rho": rho}

def display_results(greeks, option_type):
    """Display results in a simple formatted way."""
    print(f"\n{option_type.capitalize()} Option Greeks:")
    for key, value in greeks.items():
        print(f"{key:<8}: {value:.4f}")

def get_input(prompt, cast=float, positive=False):
    """Helper to get and validate input."""
    value = input(prompt)
    try:
        result = cast(value)
        if positive and result <= 0:
            raise ValueError(f"{prompt.strip()} must be positive.")
        return result
    except ValueError:
        raise ValueError(f"Invalid input for {prompt.strip()}—enter a number.")

def main():
    """Run the options calculator with streamlined inputs."""
    print("Black-Scholes Options Calculator")
    try:
        S = get_input("Stock Price (S): ", positive=True)
        K = get_input("Strike Price (K): ", positive=True)
        T = get_input("Time to Expiry (years, e.g., 1): ", positive=True)
        r = get_input("Risk-Free Rate (e.g., 0.05 for 5%): ")
        sigma = get_input("Volatility (e.g., 0.2 for 20%): ", positive=True)
        option_type = get_input("Option Type (call or put): ", cast=str).lower()

        greeks = black_scholes_greeks(S, K, T, r, sigma, option_type)
        display_results(greeks, option_type)
    except ValueError as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

Black-Scholes Options Calculator
Stock Price (S): 120
Strike Price (K): 100
Time to Expiry (years, e.g., 1): 1
Risk-Free Rate (e.g., 0.05 for 5%): 0.05
Volatility (e.g., 0.2 for 20%): .3
Option Type (call or put): call

Call Option Greeks:
Price   : 28.8804
Delta   : 0.8224
Gamma   : 0.0072
Theta   : -0.0224
Vega    : 0.3123
Rho     : 0.6980
